# 평가(기본 모델 - gemma-2b-it)

평가를 위해서 요약 결과를 확인하는 과정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import random
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments, pipeline, logging
from transformers import TrainingArguments, set_seed
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

set_seed(42)

In [ ]:
# 불러올 모델 경로 - base model
new_model = 'jental/gemma_model10000'

In [ ]:
# 허깅페이스 토큰
HUGGINGFACE_AUTH_TOKEN = ''

In [ ]:
# bnb 환경 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
    ) # 4bit quantization

In [ ]:
# 토크나이저 불러오기
# tokenizer를 먼저 불러와야 한다
tokenizer = AutoTokenizer.from_pretrained(new_model,token=HUGGINGFACE_AUTH_TOKEN, padding_side="right")

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
        new_model,
        token = HUGGINGFACE_AUTH_TOKEN,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True

adapter_config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

In [ ]:
model.eval()

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=256, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05,

In [ ]:
df_valid = pd.read_csv('/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/kobart_valid.tsv', sep='\t')

df_valid.head()

,news,summary
0,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [ ]:
df_valid.shape

(30122, 2)

In [ ]:
cnt = 0
step1 = 1000
for i in range(0, len(df_valid), step1):
    cnt += 1
    if i != 30000:
        globals()[f'part{cnt}'] = df_valid['news'][i:i+step1]
    else :
        globals()[f'part{cnt}'] = df_valid['news'][i:]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)



for data_num in range(2, 11):
    results = []
    data = globals()[f'part{data_num}']

    for text_num, text in enumerate(tqdm(data)):
        sum_len = len(df_valid['summary'][text_num])

        # prompt에 요청 할 메세지 입력
        messages = [
            { "role": "user",
            #  "content": "summarize this news to one sentence:\n\n{}".format(text)
            "content": f"Provide a summary with about one sentences of one hundred characters for the following article.:\n\n{text}"
            }
            ]

        # prompt입력을 위한 pipeline 생성
        prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # 결과물 확인
        outputs = pipe(
                prompt,
                do_sample=True,
                temperature=0.2,
                top_k=50,
                top_p=0.95,
                add_special_tokens=True
                )
        results.append({'text' : text,
                        'origin_summary' : df_valid['summary'][text_num],
                        'summary' : outputs[0]["generated_text"][len(prompt):]
                        })
    globals()[f'df_result1_part{data_num}'] = pd.DataFrame(results)

    globals()[f'df_result1_part{data_num}'].to_csv(f'/content/drive/MyDrive/new_project/project2/test_result/model1_10000/result2_{data_num}.csv', index = False)
    globals()[f'df_result1_part{data_num}'].to_excel(f'/content/drive/MyDrive/new_project/project2/test_result/model1_10000/result2_{data_num}.xlsx', index = False)

 86%|████████▌ | 861/1000 [1:52:30<24:33, 10.60s/it]